In [3]:
from dotenv import load_dotenv
from agents import Agent, trace, function_tool, Runner, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import resend
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio

In [43]:
load_dotenv(override=True)

True

In [6]:
import os

from agents import OpenAIChatCompletionsModel

GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL", "https://generativelanguage.googleapis.com/v1beta/openai/")

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GEMINI_API_KEY"))

gemini_model = OpenAIChatCompletionsModel(openai_client=gemini_client, model="gemini-2.0-flash")

In [7]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [8]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model =gemini_model
    # model=LitellmModel(
    #     model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    # ),
)

sales_agent2 = Agent(
    name="Engaging Sales Agent", instructions=instructions2, model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

sales_agent3 = Agent(
    name="Busy Sales Agent", instructions=instructions3, model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    )
)

In [9]:
result = Runner.run_streamed(sales_agent1,input="Write a cold email to a company that needs to prepare for a SOC2 audit.")
async for event in result.stream_events():
    if event.type =="raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Contact Name],

My name is [Your Name] from ComplAI. We understand that achieving and maintaining SOC 2 compliance can be a complex and resource-intensive process, and we're reaching out to offer a solution that simplifies and streamlines the entire journey.

At ComplAI, we provide an AI-powered SaaS platform specifically designed to help companies like [Company Name] prepare for and successfully navigate SOC 2 audits. Our platform offers:

*   **Automated Evidence Collection:** Reduce manual effort by automating the collection of evidence needed for audit requirements.
*   **Policy and Procedure Management:** Simplify the creation, management, and enforcement of security policies with AI-driven recommendations.
*   **Risk Assessment and Remediation:** Identify and mitigate potential security risks, ensuring a robust security posture.
*   **Continuous Monitoring:** Maintain continuous compliance through automated monitoring 

OPENAI_API_KEY is not set, skipping trace export


In [47]:
message = "Write a cold email to a company that needs to prepare for a SOC2 audit."

with trace("Parallel Cold Emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )


outputs = [result.final_output for result in results]
for output in outputs:
    print(output, "\n\n---\n\n")


Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient Name],

My name is [Your Name] and I'm with ComplAI, an AI-powered platform designed to simplify and accelerate the SOC 2 compliance process.

I understand that preparing for a SOC 2 audit can be a complex and resource-intensive undertaking. Many companies like [Company Name], particularly in the [Recipient Company Industry] sector, struggle with the manual processes involved in evidence collection, policy management, and gap analysis.

ComplAI offers a comprehensive solution that automates these tasks, significantly reducing the time and cost associated with SOC 2 compliance. Our platform:

*   **Continuously monitors your environment:** Automatically collects and analyzes evidence to demonstrate compliance with SOC 2 criteria.
*   **Provides AI-driven gap analysis:** Identifies areas where your security posture needs improvement and offers actionable remediation steps.
*   **Streamlines policy management:** Simpl

In [10]:
sales_email_picker = Agent(
    name="sales_email_picker",
    instructions="""
You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the full selected email only. ",
    """,
    model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

In [49]:
message = "Write a cold email to a company that needs to prepare for a SOC2 audit."

with trace("Parallel Cold Emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )


outputs = [result.final_output for result in results]

emails = "Cold Sales Emails:\n\n".join(outputs)

best_email = await Runner.run(
    sales_email_picker,
    emails
)
print("Best Email:\n\n", best_email.final_output, "\n\n---\n\n")

Best Email:

 Subject: Is Your SOC 2 Compliance a Ticking Time Bomb (or Just a Slightly Annoying To-Do)? 💣

Hey [Contact Name],

Let's be honest, "SOC 2 Compliance" probably isn't topping your list of fun things to do this week, is it? I imagine it's nestled somewhere between "root canal" and "untangling Christmas lights in July."

But fear not! At ComplAI, we're on a mission to make SOC 2 compliance...dare I say... *not* a soul-crushing experience. We've built an AI-powered SaaS tool that automates the tedious parts, streamlines the process, and generally makes the whole thing less likely to induce hives.

Think of us as your compliance co-pilot, navigating the audit airspace while you focus on, you know, actually building awesome stuff.

We help you:

*   **Automate Evidence Collection:** Stop the screenshot madness! (We've all been there).
*   **Identify and Remediate Gaps:** We find the skeletons in your compliance closet before the auditor does.
*   **Stay Continuously Compliant:*

In [11]:
import resend
resend.api_key = os.getenv("RESEND_API_KEY")
@function_tool
def send_email(subject:str, html_body:str):
    """Send out an email to the user with the given body."""
    params: resend.Emails.SendParams = {
        "from": "Acme <onboarding@resend.dev>",
        "to": ["adedejiosvaldo@gmail.com"],
        "subject": subject,
        "html": html_body,
    }
    email = resend.Emails.send(params)
    print(email)

In [61]:
send_email

FunctionTool(name='send_email', description='Send out an email to the user with the given body.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7c1d8e664d60>, strict_json_schema=True, is_enabled=True)

In [62]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7c1df89328e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7c1d8e643100>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [63]:
instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    tools=tools,
    model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

{'id': '3601fdc2-96ff-4b24-a6c0-3fa4ad81a3b3'}
